In [1]:
import numpy as np
import pandas as pd
from PIL import Image

/tmp/ipykernel_2193/823049271.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def get_TP_TN_FP_FN(truth,computed,positive_label):
    TP = TN = FP = FN = 0
    for i in range(len(truth)):
        if computed[i] == positive_label:
            if truth[i] == positive_label:
                TP += 1
            else:
                FP += 1
        else:
            if truth[i] == positive_label:
                FN += 1
            else:
                TN += 1

    return TP,TN,FP,FN

def getAccuracy(TP,TN,FP,FN):
    if TP+TN+FP+FN == 0:
        return 0
    return (TP + TN) / (TP+TN+FP+FN)

def getPrecision(TP,TN,FP,FN):
    if TP + FP == 0:
        return 0
    return TP/(TP + FP)

def getRecall(TP,TN,FP,FN):
    if TP+FN == 0:
        return 0
    return TP/(TP+FN)

In [7]:
from cmath import exp


class Neuron:
    def __init__(self, activation_function=lambda x: x, learning_rate = 0.001):
        self.activation_function = activation_function
        self.learning_rate = learning_rate
        self.weights = []
    
    def set_weights(self, n: int):
        for _ in range(n):
            self.weights.append(np.random.randn())
      

    def set_weights_input_layer(self):
        self.weights = [1]
    
    def proceed(self,inputs):
        return self.activation_function(sum([self.weights[i] * inputs[i] for i in range(len(inputs))]))

    def recompute_weights(self, error, input_line):
        for i in range(len(self.weights)):
            self.weights[i] = self.weights[i] - error * input_line[i] * self.learning_rate

    def get_err_part(self, error, i: int):
        total = sum([abs(self.weights[i]) for i in range(len(self.weights))])
        return error * abs(self.weights[i]) / total
        # total = sum([self.weights[i] for i in range(len(self.weights))])
        # return error * self.weights[i] / total

class ReteaNeuronala:
    def __init__(self, hidden_layers, epochs, activation_function = lambda x : x, learning_rate = 0.001) -> None:
        self.layers = [[Neuron(activation_function, learning_rate) for _ in range(layer)] for layer in hidden_layers]
        self.epochs = epochs
    
    def train(self,input_set, output_set):
        self.layers = self.layers + [[Neuron(activation_function=lambda x : 1/(1+exp(-x).real)) for _ in range(len(set(output_set)))]]

        for i in range(len(self.layers)):
            for neuron in self.layers[i]:
                if i == 0:
                    neuron.set_weights(len(input_set[-1]))
                else:
                    neuron.set_weights(len(self.layers[i-1]))

        for e in range(self.epochs):
            loss = 0
            for i in range(len(input_set)):
                input_line = input_set[i]
                output = output_set[i]
                result = self.compute(input_line)
                if output == 'YES':
                    D = 1
                    self.balance_weights(0,1-result[0],input_line, len(self.layers)-1)
                    self.balance_weights(1,0-result[1],input_line, len(self.layers)-1)
                else:
                    D = 0
                    self.balance_weights(0,0-result[0],input_line, len(self.layers)-1)
                    self.balance_weights(1,1-result[1],input_line, len(self.layers)-1)
                
                loss = loss + abs(D * np.log(result[0]) + (1-D) * np.log(1-result[0])
            
            loss = -(loss / len(input_set))
            print("Epoch {} --------> loss: {}".format(e,loss))

    def balance_weights(self,neuron_index, error, input_line, layer_index):
        self.layers[layer_index][neuron_index].recompute_weights(error,input_line)
        if layer_index != 0:
            for j in range(len(self.layers[layer_index-1])):
                err = self.layers[layer_index][neuron_index].get_err_part(error, j)
                self.balance_weights(j,err,input_line,layer_index-1)

    def compute(self, input_line):
        result = input_line
        for layer in self.layers:
            result = [layer[i].proceed(result) for i in range(len(layer))]
        return result
    
    def predict(self, inputs):
        outputs = []
        for input in inputs:
            result = self.compute(input)
            if result[0] > result[1]:
                outputs.append('YES')
            else:
                outputs.append('NO')
        return outputs

In [8]:
def read_datas() -> pd.DataFrame:
    df = pd.read_csv('datas.csv')
    df = df.dropna()

    return df

def getTrainingAndValidationDatas():
    np.random.seed(5)
    df = read_datas()
    n = df.shape[0]
    indexes = [i for i in range(n)]
    trainingIndexes = np.random.choice(indexes, int(0.7 * n), replace = False)
    validationIndexes = [i for i in range(n) if not i in trainingIndexes]

    trainingInputs = [df['Photo'].iloc[i] for i in trainingIndexes]
    trainingOutputs = [df['Has Filter'].iloc[i] for i in trainingIndexes]

    validationInputs = [df['Photo'].iloc[i] for i in validationIndexes]
    validationOutputs = [df['Has Filter'].iloc[i] for i in validationIndexes]

    return trainingInputs, trainingOutputs, validationInputs, validationOutputs

def getInputParameters(inputImages, size):
    params = []
    for imagePath in inputImages:
        params.append([1])
        image = Image.open(imagePath)
        image = image.resize(size)
        for pixel in list(image.getdata()):
            r,g,b = pixel[0],pixel[1],pixel[2]
            params[-1].append(r)
            params[-1].append(g)
            params[-1].append(b)
    return params

def tang_func(x):
    from numpy import tanh
    return tanh(x)

def defineClassifier(hidden_layers, activation_function, trainInputs, trainOutputs):
    classifier = ReteaNeuronala(hidden_layers,10,activation_function, learning_rate=0.0000001)
    classifier.train(trainInputs, trainOutputs)
    return classifier

def testClassifier(hidden_layers,activation_function,size):
    trainingInputSet, trainingOutputSet, validationInputSet, validationOutputSet = getTrainingAndValidationDatas()
    trainInputs = getInputParameters(trainingInputSet, size)
    trainOutputs = trainingOutputSet
    classifier = defineClassifier(hidden_layers, activation_function, trainInputs, trainOutputs)

    validationInputs = getInputParameters(validationInputSet, size)
    outputs = classifier.predict(validationInputs)

    TP,TN,FP,FN = get_TP_TN_FP_FN(validationOutputSet, outputs,'YES')
    acc = getAccuracy(TP,TN,FP,FN)
    pr = getPrecision(TP,TN,FP,FN)
    re = getRecall(TP,TN,FP,FN)
    print("Accuracy: {}\nPrecision: {}\nRecall: {}".format(acc,pr,re))

testClassifier([3,3,3],tang_func,(64,64))

Epoch 0 --------> loss: 0.8376231582209428
Epoch 1 --------> loss: 0.8800796982857404
Epoch 2 --------> loss: 0.8736870467770496
Epoch 3 --------> loss: 0.8585010126505775
Epoch 4 --------> loss: 0.8422965000098895
Epoch 5 --------> loss: 0.8263566473388028
Epoch 6 --------> loss: 0.810973741017731
Epoch 7 --------> loss: 0.796306447315628
Epoch 8 --------> loss: 0.7825122443928446
Epoch 9 --------> loss: 0.7697696689985537
Accuracy: 0.4074074074074074
Precision: 0.3
Recall: 0.75
